In [2]:
using Pkg
for p in ["Knet", "Plots", "IterTools","PyCall"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end
using DelimitedFiles
using Knet: KnetArray, accuracy, progress, minibatch, cycle, adam, xavier_uniform
using Plots
using IterTools: ncycle, takenth, take
using Base.Iterators: flatten
using LinearAlgebra

include("utils.jl")
include("models.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


In [3]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("cora")

(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 1.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Int32[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 1:140, 200:499, [2692, 2532, 2050, 1715, 2362, 2609, 2622, 1975, 2081, 1767  …  1885, 2305, 2354, 2135, 2601, 1770, 1995, 2504, 1749, 2157])

In [ ]:
# Row-normalize the featues
#features = features./sum(features,dims=2)

In [4]:
# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
    window_size
end

arguments = args(200, 0.01, 5e-4, 16, 0.5, 10)

# Model and optimizer
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

function val_loss(g::GCN,x,y)
    output = g(x)[:, idx_val]
    nll(output, y[idx_val]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
end  
function val_loss(g::GCN, d)
    mean(val_loss(g,x,y) for (x,y) in d)
end

function test_loss(g::GCN,x,y)
    output = g(x)[:, idx_test]
    nll(output, y[idx_test]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
end  
function test_loss(g::GCN,d)
    mean(test_loss(g,x,y) for (x,y) in d)
end

(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
#(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train])

In [5]:
output = model(features)

7×2708 Array{Float64,2}:
 -0.511534   0.175983  -1.4298   …   0.083868    -0.306011  -0.25584 
  0.285138   1.04412    1.29503      0.14618      1.1186     1.52259 
  0.155574  -0.166338  -1.68551      0.00101836   0.836543   1.84857 
 -0.445659  -0.557097  -1.91386     -0.196478    -0.591877  -1.83934 
 -0.170223   0.193604  -1.70069      0.0133759   -0.663336  -0.933878
  0.240876   0.340443   1.69431  …   0.0934175    0.375353   0.211568
  0.727045  -0.22814    1.96052      0.0526961   -0.957076  -0.104131

In [16]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]

2708-element Array{Int64,1}:
 4
 5
 5
 1
 4
 3
 1
 4
 4
 3
 1
 1
 5
 ⋮
 4
 4
 4
 4
 4
 4
 1
 4
 4
 4
 4
 4

In [17]:
nll(output[:,idx_train], labels_decoded[idx_train])

0.020875241820172585

In [18]:
accuracy(output[:,idx_train], labels_decoded[idx_train])

0.9928571428571429

In [19]:
using AutoGrad
J = @diff model(features, labels_decoded)
grad(J, params(model)[1])

16×1433 Array{Float64,2}:
 -4.94045e-6   -5.04933e-6   -4.02274e-5   …  -2.28852e-5   -9.48305e-6 
 -5.32921e-5    0.000762797  -3.16407e-5      -3.16319e-5   -2.71465e-5 
 -1.21491e-6    0.00490077   -0.00070694      -0.000197873  -3.80092e-5 
  2.29398e-8   -0.000221447   1.7864e-5        0.000137667   3.95861e-5 
 -8.44552e-5    0.00309307    0.00295061      -3.34169e-5    0.000146503
 -7.3157e-5    -8.15646e-5   -3.07072e-5   …  -7.11288e-5   -2.32561e-5 
  2.45814e-5    1.62958e-5   -0.000293657     -0.000198481   5.10374e-7 
 -1.01867e-5    4.38358e-6   -8.37259e-5       8.37628e-6    3.231e-6   
  4.49051e-6    5.14665e-5    0.000186076      1.19113e-5    0.000118167
  4.07656e-5   -6.05436e-5    2.43043e-5       0.000134306   1.2559e-5  
 -3.62687e-5    6.0946e-5    -1.94848e-5   …  -7.40089e-5    8.22732e-5 
  3.24229e-5   -5.03884e-6    0.0026383       -0.000198039  -5.49411e-6 
 -0.000154644  -0.000137142  -0.00259551      -0.000304916  -3.26745e-5 
 -7.85678e-5   -0.0001708

In [20]:
data =  minibatch(features, labels_decoded[:], length(labels_decoded))

Data{Tuple{Array{Float32,2},Array{Int64,1}}}(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [4 5 … 4 4], 2708, 2708, false, 1, 1:2708, false, (1433, 2708), (2708,), Array{Float32,2}, Array{Int64,1})

In [35]:
function train_with_results(model, data, epoch, lr)
    training = adam(model, ncycle(data, epoch), lr=lr)
    snapshot() = model(data)
    snapshots = (snapshot() for x in takenth(progress(training),length(data)))
    res = collect(flatten(snapshots))
    return res
end

train_with_results (generic function with 1 method)

In [32]:
function early_stopping(model, data, epoch, lr, window_size)
    for i=1:epoch
        training = adam(model, ncycle(data, 1), lr=lr)
        println((:epoch, i, :trnloss, model(data)), :valloss, val_loss(model, data), :tstloss, test_loss(model, data))
    end    
end

early_stopping (generic function with 1 method)

In [ ]:
early_stopping(model, data, arguments.epochs, arguments.lr, arguments.window_size)

(:epoch, 1, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 2, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 3, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 4, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 5, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 6, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 7, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 8, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 9, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 10, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.588081735991101
(:epoch, 11, :trnloss, 0.0761127856379567)valloss2.7292908158323557tstloss2.5880817359911

In [ ]:
results = train_with_results(model, data, arguments.epochs, arguments.lr)
trnloss = Array{Float32}(results)
plot(trnloss, ylim=(0.0,2.0),labels=[:trnloss],xlabel="Epochs",ylabel="Loss")

┣█████               ┫ [25.50%, 51/200, 01:04/04:10, 1.11s/i] 

In [14]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

InterruptException: InterruptException:

In [15]:
output = model(features)
accuracy(output[:,idx_test], labels_decoded[idx_test])

0.745

In [ ]:
#####################################################################################################################

In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("citeseer")

In [ ]:
# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
end

arguments = args(200, 0.01, 5e-4, 16, 0.5)

# Model and optimizer
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
#(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train])

In [ ]:
output = model(features)

In [ ]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]

In [ ]:
nll(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
using AutoGrad
J = @diff model(features, labels_decoded)
grad(J, params(model)[1])

In [ ]:
data =  minibatch(features, labels_decoded[:], length(labels_decoded))

In [ ]:
results = train_with_results(model, data, arguments.epochs, arguments.lr)
trnloss = Array{Float32}(results)
plot(trnloss, ylim=(0.0,2.0),labels=[:trnloss],xlabel="Epochs",ylabel="Loss")

In [ ]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
output = model(features)
accuracy(output[:,idx_test], labels_decoded[idx_test])

In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("cora")

In [ ]:
# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
end

arguments = args(200, 0.01, 5e-4, 16, 0.5)

# Model and optimizer
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
#(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train])

In [ ]:
output = model(features)

In [ ]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]

In [ ]:
nll(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
using AutoGrad
J = @diff model(features, labels_decoded)
grad(J, params(model)[1])

In [ ]:
data =  minibatch(features, labels_decoded[:], length(labels_decoded))

In [ ]:
results = train_with_results(model, data, arguments.epochs, arguments.lr)
trnloss = Array{Float32}(results)
plot(trnloss, ylim=(0.0,2.0),labels=[:trnloss],xlabel="Epochs",ylabel="Loss")

In [ ]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
output = model(features)
accuracy(output[:,idx_test], labels_decoded[idx_test])